# Text classification with Quanvolutional layer + Attention + HQLSTM

# Import libraries

Imports for computations

In [20]:
!pip install custatevec_cu12
!pip install pennylane pennylane-lightning-gpu
import numpy as np
import pennylane as qml
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torch.nn.functional as F
from tqdm.notebook import tqdm

For plotting data

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
sns.despine()
# plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc("axes", labelweight="bold", labelsize="large", titleweight="bold", titlepad=10)

%matplotlib inline

<Figure size 640x480 with 0 Axes>

For data preprocessing and text cleaning

In [22]:
!pip install langdetect
!pip install contractions
!pip install emoji==1.4.1
!pip install nltk

import re
import string
import emoji
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from langdetect import detect, LangDetectException
import contractions
from nltk.tokenize import word_tokenize
from collections import Counter
from gensim.models import Word2Vec

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Set seed for reproductivity

In [23]:
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

In [24]:
!pip install qulacs pennylane-qulacs

In [25]:
!pip install qiskit pennylane-qiskit

In [26]:
# from qiskit_ibm_runtime import QiskitRuntimeService
# from qiskit_ibm_provider import IBMProvider


# IBMProvider.save_account(token='d4d5a9ed2d4b576b105b9476fd9d61b464bae4f7fc0d1e543e698e576f59f537ffbf3f97a26668e5840fd0b53ab5f84541e03529de75bf2297236b8b79873dff',
#                          overwrite=True)
# service = QiskitRuntimeService()
# backend = service.backend("ibm_sherbrooke")

In [27]:
num_qubits = 4
# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "cpu"
# dev = qml.device('qulacs.simulator', wires = range(num_qubits))
# dev = qml.device("lightning.qubit", wires=range(num_qubits))
dev = qml.device('qiskit.aer', wires=range(num_qubits))
device, dev

('cpu',
 <LegacyDeviceFacade: <AerDevice device (wires=4, shots=1024) at 0x7f6b8d5f4280>>)

# Import data, data preprocessing and analysis

We use "Cyberbullying Classification" dataset from Kaggle. You can acquire more information about the data by the following link: https://www.kaggle.com/datasets/andrewmvd/cyberbullying-classification/data

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
df = pd.read_csv('cyberbullying_tweets.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'cyberbullying_tweets.csv'

In [ ]:
df

In [ ]:
df = df.rename(columns={'tweet_text': 'text', 'cyberbullying_type': 'sentiment'})

In [ ]:
df.head()

Define cleaning functions. Source: https://www.kaggle.com/code/ludovicocuoghi/detecting-bullying-tweets-pytorch-lstm-bert

In [ ]:
# Clean emojis from text
def strip_emoji(text):
    return emoji.get_emoji_regexp().sub("", text)

# Remove punctuations, stopwords, links, mentions and new line characters
def strip_all_entities(text):
    text = re.sub(r'\r|\n', ' ', text.lower())  # Replace newline and carriage return with space, and convert to lowercase
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)  # Remove links and mentions
    text = re.sub(r'[^\x00-\x7f]', '', text)  # Remove non-ASCII characters
    banned_list = string.punctuation
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

# Clean hashtags at the end of the sentence, and keep those in the middle of the sentence by removing just the # symbol
def clean_hashtags(tweet):
    # Remove hashtags at the end of the sentence
    new_tweet = re.sub(r'(\s+#[\w-]+)+\s*$', '', tweet).strip()

    # Remove the # symbol from hashtags in the middle of the sentence
    new_tweet = re.sub(r'#([\w-]+)', r'\1', new_tweet).strip()

    return new_tweet

# Filter special characters such as & and $ present in some words
def filter_chars(text):
    return ' '.join('' if ('$' in word) or ('&' in word) else word for word in text.split())

# Remove multiple spaces
def remove_mult_spaces(text):
    return re.sub(r"\s\s+", " ", text)

# Function to check if the text is in English, and return an empty string if it's not
def filter_non_english(text):
    try:
        lang = detect(text)
    except LangDetectException:
        lang = "unknown"
    return text if lang == "en" else ""

# Expand contractions
def expand_contractions(text):
    return contractions.fix(text)

# Remove numbers
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# Lemmatize words
def lemmatize(text):
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(lemmatized_words)

# Remove short words
def remove_short_words(text, min_len=2):
    words = text.split()
    long_words = [word for word in words if len(word) >= min_len]
    return ' '.join(long_words)

# Replace elongated words with their base form
def replace_elongated_words(text):
    regex_pattern = r'\b(\w+)((\w)\3{2,})(\w*)\b'
    return re.sub(regex_pattern, r'\1\3\4', text)

# Remove repeated punctuation
def remove_repeated_punctuation(text):
    return re.sub(r'[\?\.\!]+(?=[\?\.\!])', '', text)

# Remove extra whitespace
def remove_extra_whitespace(text):
    return ' '.join(text.split())

def remove_url_shorteners(text):
    return re.sub(r'(?:http[s]?://)?(?:www\.)?(?:bit\.ly|goo\.gl|t\.co|tinyurl\.com|tr\.im|is\.gd|cli\.gs|u\.nu|url\.ie|tiny\.cc|alturl\.com|ow\.ly|bit\.do|adoro\.to)\S+', '', text)

# Remove spaces at the beginning and end of the tweet
def remove_spaces_tweets(tweet):
    return tweet.strip()

# Remove short tweets
def remove_short_tweets(tweet, min_words=3):
    words = tweet.split()
    return tweet if len(words) >= min_words else ""

# Function to call all the cleaning functions in the correct order
def clean_tweet(tweet):
    tweet = strip_emoji(tweet)
    tweet = expand_contractions(tweet)
    tweet = filter_non_english(tweet)
    tweet = strip_all_entities(tweet)
    tweet = clean_hashtags(tweet)
    tweet = filter_chars(tweet)
    tweet = remove_mult_spaces(tweet)
    tweet = remove_numbers(tweet)
    tweet = lemmatize(tweet)
    tweet = remove_short_words(tweet)
    tweet = replace_elongated_words(tweet)
    tweet = remove_repeated_punctuation(tweet)
    tweet = remove_extra_whitespace(tweet)
    tweet = remove_url_shorteners(tweet)
    tweet = remove_spaces_tweets(tweet)
    tweet = remove_short_tweets(tweet)
    tweet = ' '.join(tweet.split())  # Remove multiple spaces between words
    return tweet

In [ ]:
df['text_clean'] = [clean_tweet(tweet) for tweet in df['text']]

In [ ]:
df.head()

In [ ]:
print(f'There are around {int(df["text_clean"].duplicated().sum())} duplicated tweets, we will remove them.')

In [ ]:
df.drop_duplicates("text_clean", inplace=True)

In [ ]:
df.sentiment.value_counts()

As we can see, after cleaning classes are unbalanced so we will drop "other_cyberbullying" class as there is not enough data. Later we will oversample "not_cyberbullying" class.

In [ ]:
df = df[df["sentiment"]!="other_cyberbullying"]

In [ ]:
df['text_len'] = [len(text.split()) for text in df.text_clean]

In [ ]:
plt.figure(figsize=(7,5))
ax = sns.countplot(x='text_len', data=df[df['text_len']<10], palette='mako')
plt.title('Count of tweets with less than 10 words', fontsize=20)
plt.yticks([])
ax.bar_label(ax.containers[0])
plt.ylabel('count')
plt.xlabel('')
plt.show()

In [ ]:
plt.figure(figsize=(16,5))
ax = sns.countplot(x='text_len', data=df[(df['text_len']<=1000) & (df['text_len']>10)], palette='Blues_r')
plt.title('Count of tweets with high number of words', fontsize=25)
plt.yticks([])
ax.bar_label(ax.containers[0])
plt.ylabel('count')
plt.xlabel('')
plt.show()

In [ ]:
df = df[df['text_len'] < df['text_len'].quantile(0.995)]

In [ ]:
max_len = np.max(df['text_len'])
max_len

In [ ]:
df['sentiment'] = df['sentiment'].replace({'religion':0,'age':1,'ethnicity':2,'gender':3,'not_cyberbullying':4})

Text tokenization

In [ ]:
def Tokenize(column, seq_len):
    ##Create vocabulary of words from column
    corpus = [word for text in column for word in text.split()]
    count_words = Counter(corpus)
    sorted_words = count_words.most_common()
    vocab_to_int = {w:i+1 for i, (w,c) in enumerate(sorted_words)}

    ##Tokenize the columns text using the vocabulary
    text_int = []
    for text in column:
        r = [vocab_to_int[word] for word in text.split()]
        text_int.append(r)
    ##Add padding to tokens
    features = np.zeros((len(text_int), seq_len), dtype = int)
    for i, review in enumerate(text_int):
        if len(review) <= seq_len:
            zeros = list(np.zeros(seq_len - len(review)))
            new = zeros + review
        else:
            new = review[: seq_len]
        features[i, :] = np.array(new)

    return sorted_words, features

In [ ]:
vocabulary, tokenized_column = Tokenize(df["text_clean"], max_len)

In [ ]:
X = df['text_clean']
y = df['sentiment']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Text embedding with pre-trained Word2vec model

In [ ]:
Word2vec_train_data = list(map(lambda x: x.split(), X_train))

In [ ]:
EMBEDDING_DIM = 200

In [ ]:
word2vec_model = Word2Vec(Word2vec_train_data, vector_size=EMBEDDING_DIM)

In [ ]:
VOCAB_SIZE = len(vocabulary) + 1 #+1 for the padding

In [ ]:
# Define an empty embedding matrix of shape (VOCAB_SIZE, EMBEDDING_DIM)
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))

# Fill the embedding matrix with pre-trained values from word2vec
for word, token in vocabulary:
    # Check if the word is present in the word2vec model's vocabulary
    if word in word2vec_model.wv.key_to_index:
        # If the word is present, retrieve its embedding vector and add it to the embedding matrix
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[token] = embedding_vector

# Print the shape of the embedding matrix
print("Embedding Matrix Shape:", embedding_matrix.shape)

In [ ]:
X = tokenized_column
y = df['sentiment'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

Oversampling

In [ ]:
ros = RandomOverSampler()
X_train_os, y_train_os = ros.fit_resample(np.array(X_train),np.array(y_train))

In [ ]:
X_train_os.shape

Finally, we define our preprocessed dataset and loaders

In [ ]:
train_data = TensorDataset(torch.from_numpy(X_train_os), torch.from_numpy(y_train_os))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

In [ ]:
BATCH_SIZE = 32

In [ ]:
train_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

# Implementation of the classical model

In this paragraph we will implement model with classical convolution and LSTM in order to compare it with its hybrid-quantum analogue

In [29]:
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super(Attention, self).__init__()
        # The attention linear layer which transforms the input data to the hidden space
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim )
        # The linear layer that calculates the attention scores
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        # Concatenate the last two hidden states in case of a bidirectional LSTM
        hidden = hidden[-1]
        # Repeat the hidden state across the sequence length
        hidden_repeated = hidden.unsqueeze(1).repeat(1, seq_len, 1)
        # Calculate attention weights
        attn_weights = torch.tanh(self.attn(torch.cat((hidden_repeated, encoder_outputs), dim=2)))
        # Compute attention scores
        attn_weights = self.v(attn_weights).squeeze(2)
        # Apply softmax to get valid probabilities
        return nn.functional.softmax(attn_weights, dim=1)

In [30]:
class ClassicalModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, in_channels, out_channels, kernel_size, stride, padding, hidden_dim, num_classes, lstm_layers):
        super(ClassicalModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = lstm_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, lstm_layers, batch_first=True)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x, hidden):
        # Transform words to embeddings
        embedded = self.embedding(x)
        # embedded = self.conv(embedded.unsqueeze(1))
        # # Pass embeddings to LSTM
        # out, hidden = self.lstm(embedded.reshape(-1, embedded.size(1)*embedded.size(2), embedded.size(-1)), hidden)
        out, hidden = self.lstm(embedded, hidden)
        # Calculate attention weights
        attn_weights = self.attention(hidden[0], out)
        # Calculate context vector by taking the weighted sum of LSTM outputs
        context = attn_weights.unsqueeze(1).bmm(out).squeeze(1)
        # Classify the context vector
        out = self.softmax(self.fc(context))
        return out, hidden

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return h0, c0

# Implementation of the hybrid-quantum model

In [31]:
# @qml.qnode(dev, interface='torch')
# def quanvcirc(patch, weights, wires=range(num_qubits)):
#     # Angle embedding of the patch (reshape to match expected size)
#     qml.AngleEmbedding(patch, wires=wires, rotation='Y')

#     # Apply RX rotations based on the weights
#     qml.RX(weights[0][0], wires=0)
#     qml.RX(weights[0][1], wires=1)
#     qml.RY(weights[1][0], wires=2)
#     qml.RY(weights[1][1], wires=3)

#     # Apply CNOT gates
#     qml.CNOT(wires=[0, 1])
#     qml.CNOT(wires=[1, 2])
#     qml.CNOT(wires=[2, 3])
#     qml.CNOT(wires=[3, 0])
#     # Apply RY rotations based on the weights


#     # Return the expectation values of Pauli-Z measurements on all qubits
#     return qml.expval(qml.PauliZ(0))

In [32]:
# print(qml.draw(quanvcirc)(np.ones((1, 4)), np.ones((2, 2))))

In [33]:
# class QuanConv2D(nn.Module):
#     def __init__(self, in_channels, out_channels, kernel_size, stride, padding):
#         super(QuanConv2D, self).__init__()
#         self.in_channels = in_channels
#         self.out_channels = out_channels
#         self.kernel_size = kernel_size
#         self.stride = stride
#         self.padding = padding

#         # Define weights and biases as trainable parameters
#         self.weights = nn.Parameter(
#             torch.randn(out_channels, in_channels, kernel_size, kernel_size), requires_grad = True
#         )

#     def forward(self, input):
#         # Extract dimensions
#         batch_size, in_channels, input_height, input_width = input.shape

#         # Calculate output dimensions
#         output_height = (input_height - self.kernel_size + 2 * self.padding) // self.stride + 1
#         output_width = (input_width - self.kernel_size + 2 * self.padding) // self.stride + 1

#         # Initialize output tensor
#         output = torch.zeros(batch_size, self.out_channels, output_height, output_width)

#         # If padding is required, add it to the input
#         if self.padding > 0:
#             input = F.pad(input, (self.padding, self.padding, self.padding, self.padding))

#         # Perform convolution
#         for b in range(batch_size):
#             for c_out in range(self.out_channels):
#                 for h in range(output_height):
#                     for w in range(output_width):
#                         h_start = h * self.stride
#                         h_end = h_start + self.kernel_size
#                         w_start = w * self.stride
#                         w_end = w_start + self.kernel_size

#                         # Slice the input for the current window
#                         input_slice = input[b, :, h_start:h_end, w_start:w_end]

#                         # Perform element-wise multiplication and sum with bias
#                         output[b, c_out, h, w] = quanvcirc(input_slice.reshape(1, self.kernel_size * self.kernel_size), self.weights[c_out].squeeze(0))
#         return output

In [34]:
qdi_reps1 = 2
qdi_reps2 = 2
qdi_depth = 1

In [35]:
@qml.qnode(dev, interface='torch')
def qdi_circuit(weights, input_array, wires=range(num_qubits)):
    for r in range(qdi_reps1):
        for i in range(len(wires)):
            qml.RX(weights[r][i], wires=wires[i])
        for j in range(len(wires)-1):
            qml.CNOT(wires=[wires[j], wires[j+1]])
        qml.CNOT(wires=[wires[len(wires)-1], wires[0]])
        # qml.Barrier()
    for d in range(qdi_depth):
        qml.AngleEmbedding(input_array, wires=range(num_qubits), rotation='Z')
        for r in range(qdi_reps2):
            for i in range(len(wires)):
                qml.RX(weights[qdi_reps1+d*r][i], wires=wires[i])
            for j in range(len(wires)-1):
                qml.CNOT(wires=[wires[j], wires[j+1]])
            qml.CNOT(wires=[wires[len(wires)-1], wires[0]])
            # qml.Barrier()
        # qml.Barrier()
    return [qml.expval(qml.PauliY(w)) for w in wires]

In [36]:
print(qml.draw(qdi_circuit)(np.ones((qdi_reps1 + qdi_reps2, num_qubits)), np.ones((4, 4))))

0: ──RX(1.00)─╭●───────╭X──RX(1.00)─╭●───────╭X─╭AngleEmbedding(M0)──RX(1.00)─╭●───────╭X──RX(1.00)
1: ──RX(1.00)─╰X─╭●────│───RX(1.00)─╰X─╭●────│──├AngleEmbedding(M0)──RX(1.00)─╰X─╭●────│───RX(1.00)
2: ──RX(1.00)────╰X─╭●─│───RX(1.00)────╰X─╭●─│──├AngleEmbedding(M0)──RX(1.00)────╰X─╭●─│───RX(1.00)
3: ──RX(1.00)───────╰X─╰●──RX(1.00)───────╰X─╰●─╰AngleEmbedding(M0)──RX(1.00)───────╰X─╰●──RX(1.00)

──╭●───────╭X─┤  <Y>
──╰X─╭●────│──┤  <Y>
─────╰X─╭●─│──┤  <Y>
────────╰X─╰●─┤  <Y>

M0 = 
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [37]:
qdi_circuit(np.ones((qdi_reps1 + qdi_reps2, num_qubits)), np.ones((4, 4)))

[tensor([-0.1484, -0.0703, -0.0410, -0.1094], dtype=torch.float64),
 tensor([-0.1035, -0.0391, -0.0293, -0.0508], dtype=torch.float64),
 tensor([0.1055, 0.1426, 0.1836, 0.1387], dtype=torch.float64),
 tensor([0.2676, 0.2617, 0.2441, 0.2090], dtype=torch.float64)]

In [38]:
class HQLSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, num_qubits):
        super(HQLSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size

        # Combined weights for efficiency
        self.W_input = nn.Parameter(torch.rand(4 * num_qubits, input_size), requires_grad = True).float()
        self.W_hid = nn.Parameter(torch.rand(4 * num_qubits, hidden_size), requires_grad = True).float()
        self.W_quan = nn.Parameter(torch.zeros(4, qdi_reps1 + qdi_reps2, num_qubits), requires_grad = True).float()
        self.W = nn.Parameter(torch.rand(4, hidden_size, num_qubits), requires_grad = True).float()

    def forward(self, x, hidden):
        h_prev, c_prev = hidden

        yield_input = F.linear(x, self.W_input)
        yield_hidden = F.linear(h_prev, self.W_hid)
        # Concatenate input and previous hidden state
        combined = yield_input + yield_hidden
        # combined = torch.cat((yield_input, yield_hidden), dim=1)

        # Apply linear transformation

        # Split into gates
        # combined = torch.cat([torch.stack(qdi_circuit(self.W_quan[i], combined[:, i:i*num_qubits].reshape(-1, num_qubits))) for i in range(num_qubits)], dim = 0)
        i_gate = F.linear(torch.stack(qdi_circuit(self.W_quan[0], combined[:, :num_qubits].reshape(-1, num_qubits)))[:, :, :4].T.float(), self.W[0])
        f_gate = F.linear(torch.stack(qdi_circuit(self.W_quan[1], combined[:, num_qubits:2*num_qubits].reshape(-1, num_qubits)))[:, :, :4].T.float(), self.W[1])
        g_gate = F.linear(torch.stack(qdi_circuit(self.W_quan[2], combined[:, 2*num_qubits:3*num_qubits].reshape(-1, num_qubits)))[:, :, :4].T.float(), self.W[2])
        o_gate = F.linear(torch.stack(qdi_circuit(self.W_quan[3], combined[:, 3*num_qubits:4*num_qubits].reshape(-1, num_qubits)))[:, :, :4].T.float(), self.W[3])



        # combined = torch.cat((i_gate, f_gate, g_gate, o_gate), dim=0).float()
        # gates = F.linear(combined, self.W)

        # # Split into gates
        # i_gate, f_gate, g_gate, o_gate = gates.chunk(4, 0)

        # Apply non-linearities
        i_gate = torch.sigmoid(i_gate)
        f_gate = torch.sigmoid(f_gate)
        g_gate = torch.tanh(g_gate)
        o_gate = torch.sigmoid(o_gate)

        # Update cell state
        c_next = (f_gate * c_prev) + (i_gate * g_gate)

        # Update hidden state
        h_next = o_gate * torch.tanh(c_next)

        return h_next, c_next


class HQLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_qubits, num_layers=1):
        super(HQLSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm_cells = nn.ModuleList([HQLSTMCell(input_size if l==0 else hidden_size, hidden_size, num_qubits) for l in range(num_layers)])


    def forward(self, x, hidden=None):
        batch_size, seq_len, _ = x.size()

        if hidden is None:
             h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
             c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        else:
            h0, c0 = hidden

        output_seq = []

        for t in range(seq_len):
            input_t = x[:, t, :] # input at current timestep
            # print(f'step:{t}/{seq_len}, hidden:')

            new_h = []
            new_c = []

            for layer in range(self.num_layers):

                # Get hidden state for current layer
                h_t, c_t = self.lstm_cells[layer](input_t, (h0[layer], c0[layer]))

                # Update hidden states for next timestep, for current layer
                new_h.append(h_t)
                new_c.append(c_t)
                input_t = h_t # The output of current layer is input for the next

            h0 = torch.stack(new_h)
            c0 = torch.stack(new_c)
            # Append hidden state at the topmost layer
            output_seq.append(h_t.unsqueeze(1))

        # Concatenate the outputs over the sequence length
        output_seq = torch.cat(output_seq, dim=1) # output_seq is of shape (batch_size, seq_len, hidden_size)

        return output_seq, (h0, c0)


Toy hybrid-quantum model for benchmarking

In [39]:
class ToyHQModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, in_channels, out_channels, kernel_size, stride, padding, hidden_dim, num_qubits, num_classes, lstm_layers):
        super(ToyHQModel, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.num_layers = lstm_layers
        self.hidden_dim = hidden_dim

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.attention = Attention(hidden_dim)
        # self.conv = QuanConv2D(self.in_channels, self.out_channels, self.kernel_size, self.stride, self.padding)
        self.lstm = HQLSTM(embedding_dim, hidden_dim, num_qubits, lstm_layers)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Transform words to embeddings
        embedded = self.embedding(x)
        # x = self.conv(x.unsqueeze(1)).to(device)
        # Pass embeddings to LSTM
        out, hidden = self.lstm(embedded)
        # Calculate attention weights
        attn_weights = self.attention(hidden[0], out)
        # Calculate context vector by taking the weighted sum of LSTM outputs
        context = attn_weights.unsqueeze(1).bmm(out).squeeze(1)
        # Classify the context vector
        out = self.softmax(self.fc(context))
        return out

    def init_hidden(self, batch_size):
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_dim).to(device)
        return h0, c0

In [40]:
class HybridQuantumModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, in_channels, out_channels, kernel_size, stride, padding, hidden_dim, num_qubits, num_classes, lstm_layers):
        super(HybridQuantumModel, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = HQLSTM(embedding_dim, hidden_dim, num_qubits, lstm_layers)
        self.attention = Attention(hidden_dim)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        # Transform words to embeddings
        embedded = self.embedding(x)
        # embedded = self.conv(embedded.unsqueeze(1)).to(device)
        # Pass embeddings to LSTM
        # out, hidden = self.lstm(embedded.reshape(-1, embedded.size(1)*embedded.size(2), embedded.size(-1)))
        out, hidden = self.lstm(embedded)
        # Calculate attention weights
        attn_weights = self.attention(hidden[0], out)
        # Calculate context vector by taking the weighted sum of LSTM outputs
        context = attn_weights.unsqueeze(1).bmm(out).squeeze(1)
        # Classify the context vector
        out = self.softmax(self.fc(context))
        return out

# Training the models

In [104]:
NUM_CLASSES = 5 #We are dealing with a multiclass classification of 5 classes
HIDDEN_DIM = 100 #number of neurons of the internal state (internal neural network in the LSTM)
LSTM_LAYERS = 1 #Number of stacked LSTM layers
IN_CHANNELS = 1
OUT_CHANNELS = 5
KERNEL_SIZE = 2
STRIDE = 1
PADDING = 1


LR = 8e-4 #Learning rate
EPOCHS = 10 #Number of training epoch

criterion = nn.NLLLoss()

In [105]:
classical_model = ClassicalModel(VOCAB_SIZE, EMBEDDING_DIM, IN_CHANNELS, OUT_CHANNELS, KERNEL_SIZE, STRIDE, PADDING, HIDDEN_DIM, NUM_CLASSES, LSTM_LAYERS)
classical_model = classical_model.to(device)
# classical_model = torch.compile(classical_model)

cl_optimizer = torch.optim.AdamW(classical_model.parameters(), lr=LR, weight_decay = 5e-6)

print(classical_model)

ClassicalModel(
  (embedding): Embedding(37575, 200)
  (conv): Conv2d(1, 5, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
  (lstm): LSTM(200, 100, batch_first=True)
  (attention): Attention(
    (attn): Linear(in_features=200, out_features=100, bias=True)
    (v): Linear(in_features=100, out_features=1, bias=False)
  )
  (fc): Linear(in_features=100, out_features=5, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [106]:
def initialize_weights(model):
    for name, param in model.named_parameters():
        if 'embedding' in name:
            nn.init.uniform_(param, -0.1, 0.1)
        elif 'lstm' in name:
            if 'weight' in name:
                nn.init.orthogonal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0)
        elif 'fc' in name:
            nn.init.xavier_uniform_(param)

In [107]:
hybrid_quantum_model = HybridQuantumModel(VOCAB_SIZE, EMBEDDING_DIM, IN_CHANNELS, OUT_CHANNELS, KERNEL_SIZE, STRIDE, PADDING, HIDDEN_DIM, num_qubits, NUM_CLASSES, LSTM_LAYERS)
hybrid_quantum_model = hybrid_quantum_model.to(device)
# initialize_weights(hybrid_quantum_model)


# hybrid_quantum_model = torch.compile(hybrid_quantum_model)

hq_optimizer = torch.optim.AdamW(hybrid_quantum_model.parameters(), lr=LR, weight_decay = 5e-6)

print(hybrid_quantum_model)

HybridQuantumModel(
  (embedding): Embedding(37575, 200)
  (lstm): HQLSTM(
    (lstm_cells): ModuleList(
      (0): HQLSTMCell()
    )
  )
  (attention): Attention(
    (attn): Linear(in_features=200, out_features=100, bias=True)
    (v): Linear(in_features=100, out_features=1, bias=False)
  )
  (fc): Linear(in_features=100, out_features=5, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [108]:
import time

def train(model, criterion, optimizer,
          train_dataloader, test_dataloader, num_epochs):

    train_losses = np.zeros(num_epochs)
    test_losses = np.zeros(num_epochs)

    train_accuracy_arr = np.zeros(num_epochs)
    test_accuracy_arr = np.zeros(num_epochs)

    for i_epoch in tqdm(range(num_epochs)):
        it = 0
        train_loss = 0
        test_loss = 0

        train_accuracy = 0
        test_accuracy = 0

        # train step
        model.train()
        for batch in train_dataloader:
            start_time = time.time()
            X = batch[0].to(device)
            y = batch[1].to(device)

            h = model.init_hidden(y.size(0))
            # model forward-pass
            preds, h = model(X, h)

            # model backward-pass
            optimizer.zero_grad() # t.grad = torch.tensor([0., 0., 0.])
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step() # t = t - lr * t.grad
            end_time = time.time()
            execution_time = end_time - start_time

            # save loss and accuracy
            train_loss += loss.detach().cpu().numpy()
            print(f'batch: {it+1}/{len(train_dataloader)}, loss: {train_loss/(it+1):.4f}, time: {execution_time:.4f}')
            it += 1
            train_accuracy += (preds.argmax(-1).detach() == y).cpu().numpy().mean()


        train_loss /= len(train_dataloader)
        train_accuracy /= len(train_dataloader)
        train_losses[i_epoch] = train_loss
        train_accuracy_arr[i_epoch] = train_accuracy

        # test step
        model.eval()
        for batch in test_dataloader:
            X = batch[0].to(device)
            y = batch[1].to(device)

            h = model.init_hidden(y.size(0))

            with torch.no_grad():
                # model forward-pass
                preds, h = model(X, h)
                loss = criterion(preds, y)

                # save loss and accuracy
                test_loss += loss.detach().cpu().numpy()
                test_accuracy += (preds.argmax(-1) == y).cpu().numpy().mean()


        test_loss /= len(test_dataloader)
        test_accuracy /= len(test_dataloader)

        test_losses[i_epoch] = test_loss
        test_accuracy_arr[i_epoch] = test_accuracy

    return train_losses, test_losses, train_accuracy_arr, test_accuracy_arr

def train_for_hqnn(model, criterion, optimizer,
          train_dataloader, test_dataloader, num_epochs):

    train_losses = np.zeros(num_epochs)
    test_losses = np.zeros(num_epochs)

    train_accuracy_arr = np.zeros(num_epochs)
    test_accuracy_arr = np.zeros(num_epochs)

    for i_epoch in tqdm(range(num_epochs)):
        it = 0

        train_loss = 0
        test_loss = 0

        train_accuracy = 0
        test_accuracy = 0

        # train step
        model.train()
        for batch in train_dataloader:
            start_time = time.time()
            X = batch[0].to(device)
            y = batch[1].to(device)

            # model forward-pass
            preds = model(X)

            # model backward-pass
            optimizer.zero_grad() # t.grad = torch.tensor([0., 0., 0.])
            loss = criterion(preds, y)
            loss.backward()
            optimizer.step() # t = t - lr * t.grad
            end_time = time.time()
            execution_time = end_time - start_time

            # save loss and accuracy
            train_loss += loss.detach().cpu().numpy()
            print(f'it: {it+1}/{len(train_dataloader)}, loss: {train_loss/(it+1):.4f}, time: {execution_time:.4f}')
            it += 1
            train_accuracy += (preds.argmax(-1).detach() == y).cpu().numpy().mean()


        train_loss /= len(train_dataloader)
        train_accuracy /= len(train_dataloader)
        train_losses[i_epoch] = train_loss

        train_accuracy_arr[i_epoch] = train_accuracy

        # test step
        model.eval()
        for batch in test_dataloader:
            X = batch[0].to(device)
            y = batch[1].to(device)


            with torch.no_grad():
                # model forward-pass
                preds = model(X)
                loss = criterion(preds, y)

                # save loss and accuracy
                test_loss += loss.detach().cpu().numpy()
                test_accuracy += (preds.argmax(-1) == y).cpu().numpy().mean()


        test_loss /= len(test_dataloader)
        test_accuracy /= len(test_dataloader)

        test_losses[i_epoch] = test_loss
        test_accuracy_arr[i_epoch] = test_accuracy

    return train_losses, test_losses, train_accuracy_arr, test_accuracy_arr

In [ ]:
cl_train_losses, \
    cl_test_losses, \
    cl_train_accuracy_arr, \
    cl_test_accuracy_arr = train(classical_model, criterion=criterion,
                              optimizer=cl_optimizer,
                              train_dataloader=train_loader,
                              test_dataloader=test_loader,
                              num_epochs=EPOCHS)

  0%|          | 0/10 [00:00<?, ?it/s]

batch: 1/979, loss: 1.6025, time: 0.1276
batch: 2/979, loss: 1.5963, time: 0.0033
batch: 3/979, loss: 1.5963, time: 0.0028
batch: 4/979, loss: 1.5950, time: 0.0028
batch: 5/979, loss: 1.5930, time: 0.0028
batch: 6/979, loss: 1.6064, time: 0.0027
batch: 7/979, loss: 1.6050, time: 0.0026
batch: 8/979, loss: 1.6055, time: 0.0027
batch: 9/979, loss: 1.5994, time: 0.0027
batch: 10/979, loss: 1.5931, time: 0.0027
batch: 11/979, loss: 1.5954, time: 0.0028
batch: 12/979, loss: 1.5932, time: 0.0027
batch: 13/979, loss: 1.5869, time: 0.0026
batch: 14/979, loss: 1.5778, time: 0.0028
batch: 15/979, loss: 1.5739, time: 0.0027
batch: 16/979, loss: 1.5638, time: 0.0028
batch: 17/979, loss: 1.5614, time: 0.0027
batch: 18/979, loss: 1.5603, time: 0.0027
batch: 19/979, loss: 1.5515, time: 0.0026
batch: 20/979, loss: 1.5483, time: 0.0028
batch: 21/979, loss: 1.5448, time: 0.0031
batch: 22/979, loss: 1.5425, time: 0.0028
batch: 23/979, loss: 1.5357, time: 0.0027
batch: 24/979, loss: 1.5311, time: 0.0026
b

KeyboardInterrupt: 

In [ ]:
cl_test_accuracy_arr

In [109]:
hq_train_losses, \
    hq_test_losses, \
    hq_train_accuracy_arr, \
    hq_test_accuracy_arr = train_for_hqnn(hybrid_quantum_model, criterion=criterion,
                              optimizer=hq_optimizer,
                              train_dataloader=train_loader,
                              test_dataloader=test_loader,
                              num_epochs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
hq_test_accuracy_arr

In [ ]:
hq_train_accuracy_arr

In [ ]:
batch_size = 2
in_channels = 1
out_channels = 3
length = 31
num_classes = 5
kernel_size = 2
embedding_dim = 200
hidden_dim = 100
padding = 1
stride = 1
num_qubits = 4

# Create dummy data & labels
train_dummy_data = torch.randint(0, 1000, (batch_size, length)).to(device)
train_dummy_labels = torch.randint(0, num_classes, (batch_size,)).to(device)

test_dummy_data = torch.randn(batch_size, length)
test_dummy_labels = torch.randint(0, num_classes, (batch_size,))

# Create a simple Dataset and DataLoader
train_dummy_dataset = TensorDataset(train_dummy_data, train_dummy_labels)
train_dummy_loader = DataLoader(train_dummy_dataset, batch_size)

test_dummy_dataset = TensorDataset(test_dummy_data, test_dummy_labels)
test_dummy_loader = DataLoader(test_dummy_dataset, batch_size)


In [ ]:
LR = 8e-4 #Learning rate

toy_hq_model = ToyHQModel(VOCAB_SIZE, embedding_dim, in_channels, out_channels, kernel_size, stride, padding, hidden_dim, num_qubits, num_classes, lstm_layers = 1)

toy_hq_model = toy_hq_model.to(device)
# Set up the criterion (loss function)
criterion = nn.NLLLoss()
optimizer = torch.optim.AdamW(toy_hq_model.parameters(), lr=LR, weight_decay = 5e-6)

print(toy_hq_model)

ToyHQModel(
  (embedding): Embedding(37569, 200)
  (attention): Attention(
    (attn): Linear(in_features=200, out_features=100, bias=True)
    (v): Linear(in_features=100, out_features=1, bias=False)
  )
  (lstm): HQLSTM(
    (lstm_cells): ModuleList(
      (0): HQLSTMCell()
    )
  )
  (fc): Linear(in_features=100, out_features=5, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [ ]:
toy_hq_model(train_dummy_data)

torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])
torch.Size([2, 16])


tensor([[-1.6381, -1.6118, -1.5608, -1.6813, -1.5606],
        [-1.6472, -1.6141, -1.5558, -1.6795, -1.5565]], device='cuda:0',
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
print(X_train.shape)

(28985, 31)


In [ ]:
# train_dummy_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
# test_dummy_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

In [ ]:
# BATCH_SIZE = 2

In [ ]:
# train_dummy_loader = DataLoader(train_data, shuffle=True, batch_size=BATCH_SIZE, drop_last=True)
test_dummy_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE, drop_last=True)

In [ ]:
train_losses, \
    test_losses, \
    train_accuracy_arr, \
    test_accuracy_arr = train_for_hqnn(toy_hq_model, criterion=criterion,
                              optimizer=optimizer,
                              train_dataloader=train_dummy_loader,
                              test_dataloader=test_dummy_loader,
                              num_epochs=2)

  0%|          | 0/2 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_losses